# Fetching Weather Data and Uploading to AWS S3

This guide provides steps for fetching weather data using the OpenWeatherMap API and then uploading the data to an Amazon S3 bucket using Python.

## Step 1: Obtain an API Key from OpenWeatherMap

1. Register on the [OpenWeatherMap website](https://openweathermap.org/) and create an account.
2. Find and copy the API key from your account dashboard.

## Step 2: Write Python Function to Fetch Weather Data

In [180]:
import requests

def get_weather_data(city, api_key):
    base_url = "https://api.openweathermap.org/data/2.5/weather"
    params = {'q': city, 'appid': api_key}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Failed to fetch weather data")

## Step 3: Set Up AWS Credentials for S3 Access
* Install Boto3 using pip install boto3.
* Configure AWS credentials (AWS Access Key ID and Secret Access Key).

In [181]:
!pip install boto3

## Step 4: Write Python Function to Upload Data to S3


In [182]:
import boto3
import json
import pandas as pd

def upload_to_s3(bucket_name, file_name, data):
    s3 = boto3.client('s3')
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=json.dumps(data))

## Step 5: Combine the Functions in a Script


In [183]:
# Main execution
api_key = "9d205ed8d6a08d68d34b0bcac606ec6f"   # Replace with your API key
city = "London"  # Replace with desired city
bucket_name = "lab-03-group3"  # Replace with your S3 bucket name
file_name = "Group_3_weather_data_london.json"

try:
    weather_data = get_weather_data(city, api_key)
    upload_to_s3(bucket_name, file_name, weather_data)
    print("Data uploaded successfully to S3")
except Exception as e:
    print(f"An error occurred: {e}")

Data uploaded successfully to S3


In [184]:
# Example usage
try:
    weather_data = get_weather_data(city, api_key)
    print(weather_data)
except Exception as e:
    print(f"An error occurred: {e}")

{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 285.85, 'feels_like': 285.4, 'temp_min': 284.64, 'temp_max': 286.91, 'pressure': 1005, 'humidity': 85}, 'visibility': 10000, 'wind': {'speed': 3.13, 'deg': 246, 'gust': 5.81}, 'rain': {'1h': 0.45}, 'clouds': {'all': 100}, 'dt': 1712258850, 'sys': {'type': 2, 'id': 2075535, 'country': 'GB', 'sunrise': 1712208487, 'sunset': 1712255896}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


# Assignment: Groups to Convert JSON to CSV

Step 1. Take the JSON output and convert it to a Dataframe using pandas
Step 2. Now upload the CSV file to the 'lab-03' S3 bucket in the cloud with the following naming convention: <your group name>_weather_date_london_<datetimestamp>.csv


In [185]:
# Generic function to validate the data loaded into S3 bucket
def print_bucket_contents(bucket):
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket)

    if 'Contents' in response:
        print("Contents of the '{}' bucket:".format(bucket))
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("The bucket '{}' is empty.".format(bucket))

In [186]:
# Configure to display all the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Converting json to dataframe
flattened_data = pd.json_normalize(weather_data) # Flatten the JSON data

# Flatten 'weather' column separately with prefix
weather_data_norm = pd.json_normalize(weather_data['weather']).add_prefix('weather.')

# Merge the two DataFrames
df = pd.concat([flattened_data, weather_data_norm], axis=1)
df = df.drop(columns = ['weather'], axis = 1) # drop the weather column

# Display DataFrame
df

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.humidity,wind.speed,wind.deg,wind.gust,rain.1h,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset,weather.id,weather.main,weather.description,weather.icon
0,stations,10000,1712258850,3600,2643743,London,200,-0.1257,51.5085,285.85,285.4,284.64,286.91,1005,85,3.13,246,5.81,0.45,100,2,2075535,GB,1712208487,1712255896,500,Rain,light rain,10n


In [187]:
file_name_csv = "Group_3_weather_data_london.csv"

# Convert DataFrame to CSV string
try:
    csv_content = df.to_csv(index=False)
except Exception as e:
    print("Error converting DataFrame to CSV:", e)
    csv_content = None

# Upload the CSV content to S3
if csv_content is not None:
    try:
        s3 = boto3.client('s3')
        response = s3.put_object(
            Bucket=bucket_name,
            Key=file_name_csv,
            Body=csv_content.encode('utf-8')  # Convert to bytes
        )
        print("Data uploaded successfully to S3")
    except Exception as e:
        print("Error uploading file to S3:", e)
else:
    print("CSV content is None. Upload aborted.")

Data uploaded successfully to S3


In [188]:
# Validating if json file is uploaded in respective bucket or not
print_bucket_contents(bucket_name)

Contents of the 'lab-03-group3' bucket:
Group_3_weather_data_london.csv
Group_3_weather_data_london.json
